In [1]:
import ccxt
import time
import os
import datetime
import numpy as np
import pandas as pd

msec = 1000
minute = 60 * msec
hour = 60 * minute

symbol = 'ETH/USDT'
symbol_ = 'ETH_USDT'
market = 'binance'
timewindow = '1h'

if timewindow == '1h':
    offset = hour
    delay = offset / 1000
elif timewindow == '1m':
    offset = minute
    delay = offset / 1000
    
elif timewindow == '5m':
    offset = 5 * minute
    delay = offset / 1000

In [2]:
def get_file_contents(filename):
    try:
        with open(filename, 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)


binance = ccxt.binance()
home_path = os.path.expanduser("~")
binance.apiKey = get_file_contents(home_path + '/api_key/binance/pub')
binance.secret = get_file_contents(home_path + '/api_key/binance/private')

In [3]:
binance = ccxt.binance({
    'apiKey': binance.apiKey,
    'secret': binance.secret,
})
exchange = binance

In [4]:
from_datetime = '2017-01-01 00:00:00'
from_timestamp = exchange.parse8601(from_datetime)

# to_datetime = '2017-08-17 05:00:00'
to_datetime = '2019-09-01 00:00:00'
to_timestamp = exchange.parse8601(to_datetime)

In [5]:
t = exchange.fetch_trades(symbol=symbol, since=from_timestamp)

In [6]:
        # aggregate trades
        #
        #     [
        #         {
        #             "a": 26129,         # Aggregate tradeId
        #             "p": "0.01633102",  # Price
        #             "q": "4.70443515",  # Quantity
        #             "f": 27781,         # First tradeId
        #             "l": 27781,         # Last tradeId
        #             "T": 1498793709153, # Timestamp
        #             "m": True,          # Was the buyer the maker?
        #             "M": True           # Was the trade the best price match?
        #         }
        #     ]
        #

In [7]:
buy_amount_list = []
sell_amount_list = []
for trade in t:
    if trade.get('side') == 'buy':
        buy_amount_list.append(trade.get('amount'))

    elif trade.get('side') == 'sell':
        sell_amount_list.append(trade.get('amount'))

buy_amount_np = np.array(buy_amount_list)
sell_amount_np = np.array(sell_amount_list)
N_buy = len(buy_amount_np)
N_sell = len(sell_amount_list)
buy_amount_avg = np.mean(buy_amount_np)
sell_amount_avg = np.mean(sell_amount_np)
buy_amount_std = np.std(buy_amount_np)
sell_amount_std = np.std(sell_amount_np)

/home/nam/miniconda3/envs/python3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/nam/miniconda3/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/nam/miniconda3/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/nam/miniconda3/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/nam/miniconda3/envs/python3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
df = pd.DataFrame()

np_df = np.zeros([300000 , 16])
header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']

Total_buy_time_empty = 0
Total_sell_time_empty = 0
import time
start = time.time()

idx = 0
while from_timestamp < to_timestamp:
    try:
        ohlcvs = exchange.fetch_ohlcv(symbol, timewindow, from_timestamp, limit=1)
        
        # 1 group of trade is:
        t = exchange.fetch_trades(symbol=symbol, since=from_timestamp)
        buy_amount_list = []
        sell_amount_list = []
        price_list = []
        cost_list = []
        for trade in t:
            if trade.get('side') == 'buy':
                buy_amount_list.append(trade.get('amount'))

            elif trade.get('side') == 'sell':
                sell_amount_list.append(trade.get('amount'))
            
            price_list.append(trade.get('price'))
            cost_list.append(trade.get('cost'))
            

#         assert buy_amount_list != None, print('Not having any buy')
#         assert sell_amount_list != None, print('Not having any sell')
        if buy_amount_list == []:
            print('Not having any buyer')
            Total_buy_time_empty +=1
            buy_amount_list = [-1.0]
            
        if sell_amount_list == []:
            print('Not having any sell')
            Total_sell_time_empty +=1
            buy_amount_list = [-1.0]
            
        if price_list == []:
            price_list = [-1.0]
            
        if cost_list == []:
            cost_list = [-1.0]

        buy_amount_np = np.array(buy_amount_list)
        sell_amount_np = np.array(sell_amount_list)
        price_np = np.array(price_list)
        cost_np = np.array(cost_list)
            
        df_current = pd.DataFrame(ohlcvs, columns=header)
                
        df_current['N_buy'] = len(buy_amount_np)
        df_current['N_sell'] = len(sell_amount_list)
        df_current['buy_amount_avg'] = np.mean(buy_amount_np)
        df_current['sell_amount_avg'] = np.mean(sell_amount_np)
        df_current['buy_amount_std'] = np.std(buy_amount_np)
        df_current['sell_amount_std'] = np.std(sell_amount_np)
 
        df_current['price_avg'] = np.mean(price_np)
        df_current['price_std'] = np.std(price_np)
        
        df_current['cost_avg'] = np.mean(cost_np)
        df_current['cost_std'] = np.std(cost_np)
#         df = df.append(df_current, ignore_index=True)

#         df_array.append(df_current.values[0])
    
        np_df[idx] =df_current.values[0]
        idx += 1

#         print(exchange.milliseconds(), 'Fetched', len(ohlcvs), 'candles')
        if len(ohlcvs) > 0:
            first = ohlcvs[0][0]
            last = ohlcvs[-1][0]
#             print('First candle epoch', first, exchange.iso8601(first))

            from_timestamp = ohlcvs[-1][0] + offset
            # v = ohlcvs[0][0]/ 1000
            # !date --date @{v} +"%Y-%m-%d %H:%M"
            print('Last candle epoch', last, exchange.iso8601(last))
        else:
            break

    except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:
        print('Got an error', type(error).__name__,
              error.args, ', retrying in', offset, 'seconds...')
        
done = time.time()
elapsed = done - start
print(elapsed)

Not having any buyer
Not having any sell
Last candle epoch 1502942400000 2017-08-17T04:00:00.000Z
Last candle epoch 1502946000000 2017-08-17T05:00:00.000Z
Last candle epoch 1502949600000 2017-08-17T06:00:00.000Z
Last candle epoch 1502953200000 2017-08-17T07:00:00.000Z
Last candle epoch 1502956800000 2017-08-17T08:00:00.000Z
Last candle epoch 1502960400000 2017-08-17T09:00:00.000Z
Last candle epoch 1502964000000 2017-08-17T10:00:00.000Z
Last candle epoch 1502967600000 2017-08-17T11:00:00.000Z
Last candle epoch 1502971200000 2017-08-17T12:00:00.000Z
Last candle epoch 1502974800000 2017-08-17T13:00:00.000Z
Last candle epoch 1502978400000 2017-08-17T14:00:00.000Z
Last candle epoch 1502982000000 2017-08-17T15:00:00.000Z
Last candle epoch 1502985600000 2017-08-17T16:00:00.000Z
Last candle epoch 1502989200000 2017-08-17T17:00:00.000Z
Last candle epoch 1502992800000 2017-08-17T18:00:00.000Z
Last candle epoch 1502996400000 2017-08-17T19:00:00.000Z
Last candle epoch 1503000000000 2017-08-17T20:0

Last candle epoch 1503457200000 2017-08-23T03:00:00.000Z
Last candle epoch 1503460800000 2017-08-23T04:00:00.000Z
Last candle epoch 1503464400000 2017-08-23T05:00:00.000Z
Last candle epoch 1503468000000 2017-08-23T06:00:00.000Z
Last candle epoch 1503471600000 2017-08-23T07:00:00.000Z
Last candle epoch 1503475200000 2017-08-23T08:00:00.000Z
Last candle epoch 1503478800000 2017-08-23T09:00:00.000Z
Last candle epoch 1503482400000 2017-08-23T10:00:00.000Z
Last candle epoch 1503486000000 2017-08-23T11:00:00.000Z
Last candle epoch 1503489600000 2017-08-23T12:00:00.000Z
Last candle epoch 1503493200000 2017-08-23T13:00:00.000Z
Last candle epoch 1503496800000 2017-08-23T14:00:00.000Z
Last candle epoch 1503500400000 2017-08-23T15:00:00.000Z
Last candle epoch 1503504000000 2017-08-23T16:00:00.000Z
Last candle epoch 1503507600000 2017-08-23T17:00:00.000Z
Last candle epoch 1503511200000 2017-08-23T18:00:00.000Z
Last candle epoch 1503514800000 2017-08-23T19:00:00.000Z
Last candle epoch 1503518400000

Last candle epoch 1503979200000 2017-08-29T04:00:00.000Z
Last candle epoch 1503982800000 2017-08-29T05:00:00.000Z
Last candle epoch 1503986400000 2017-08-29T06:00:00.000Z
Last candle epoch 1503990000000 2017-08-29T07:00:00.000Z
Last candle epoch 1503993600000 2017-08-29T08:00:00.000Z
Last candle epoch 1503997200000 2017-08-29T09:00:00.000Z
Last candle epoch 1504000800000 2017-08-29T10:00:00.000Z
Last candle epoch 1504004400000 2017-08-29T11:00:00.000Z
Last candle epoch 1504008000000 2017-08-29T12:00:00.000Z
Last candle epoch 1504011600000 2017-08-29T13:00:00.000Z
Last candle epoch 1504015200000 2017-08-29T14:00:00.000Z
Last candle epoch 1504018800000 2017-08-29T15:00:00.000Z
Last candle epoch 1504022400000 2017-08-29T16:00:00.000Z
Last candle epoch 1504026000000 2017-08-29T17:00:00.000Z
Last candle epoch 1504029600000 2017-08-29T18:00:00.000Z
Last candle epoch 1504033200000 2017-08-29T19:00:00.000Z
Last candle epoch 1504036800000 2017-08-29T20:00:00.000Z
Last candle epoch 1504040400000

Last candle epoch 1504497600000 2017-09-04T04:00:00.000Z
Last candle epoch 1504501200000 2017-09-04T05:00:00.000Z
Last candle epoch 1504504800000 2017-09-04T06:00:00.000Z
Last candle epoch 1504508400000 2017-09-04T07:00:00.000Z
Last candle epoch 1504512000000 2017-09-04T08:00:00.000Z
Last candle epoch 1504515600000 2017-09-04T09:00:00.000Z
Last candle epoch 1504519200000 2017-09-04T10:00:00.000Z
Last candle epoch 1504522800000 2017-09-04T11:00:00.000Z
Last candle epoch 1504526400000 2017-09-04T12:00:00.000Z
Last candle epoch 1504530000000 2017-09-04T13:00:00.000Z
Last candle epoch 1504533600000 2017-09-04T14:00:00.000Z
Last candle epoch 1504537200000 2017-09-04T15:00:00.000Z
Last candle epoch 1504540800000 2017-09-04T16:00:00.000Z
Last candle epoch 1504544400000 2017-09-04T17:00:00.000Z
Last candle epoch 1504548000000 2017-09-04T18:00:00.000Z
Last candle epoch 1504551600000 2017-09-04T19:00:00.000Z
Last candle epoch 1504555200000 2017-09-04T20:00:00.000Z
Last candle epoch 1504558800000

Last candle epoch 1505037600000 2017-09-10T10:00:00.000Z
Last candle epoch 1505041200000 2017-09-10T11:00:00.000Z
Last candle epoch 1505044800000 2017-09-10T12:00:00.000Z
Last candle epoch 1505048400000 2017-09-10T13:00:00.000Z
Last candle epoch 1505052000000 2017-09-10T14:00:00.000Z
Last candle epoch 1505055600000 2017-09-10T15:00:00.000Z
Last candle epoch 1505059200000 2017-09-10T16:00:00.000Z
Last candle epoch 1505062800000 2017-09-10T17:00:00.000Z
Last candle epoch 1505066400000 2017-09-10T18:00:00.000Z
Last candle epoch 1505070000000 2017-09-10T19:00:00.000Z
Last candle epoch 1505073600000 2017-09-10T20:00:00.000Z
Last candle epoch 1505077200000 2017-09-10T21:00:00.000Z
Last candle epoch 1505080800000 2017-09-10T22:00:00.000Z
Last candle epoch 1505084400000 2017-09-10T23:00:00.000Z
Last candle epoch 1505088000000 2017-09-11T00:00:00.000Z
Last candle epoch 1505091600000 2017-09-11T01:00:00.000Z
Last candle epoch 1505095200000 2017-09-11T02:00:00.000Z
Last candle epoch 1505098800000

...

Last candle epoch 1564938000000 2019-08-04T17:00:00.000Z
Last candle epoch 1564941600000 2019-08-04T18:00:00.000Z
Last candle epoch 1564945200000 2019-08-04T19:00:00.000Z
Last candle epoch 1564948800000 2019-08-04T20:00:00.000Z
Last candle epoch 1564952400000 2019-08-04T21:00:00.000Z
Last candle epoch 1564956000000 2019-08-04T22:00:00.000Z
Last candle epoch 1564959600000 2019-08-04T23:00:00.000Z
Last candle epoch 1564963200000 2019-08-05T00:00:00.000Z
Last candle epoch 1564966800000 2019-08-05T01:00:00.000Z
Last candle epoch 1564970400000 2019-08-05T02:00:00.000Z
Last candle epoch 1564974000000 2019-08-05T03:00:00.000Z
Last candle epoch 1564977600000 2019-08-05T04:00:00.000Z
Last candle epoch 1564981200000 2019-08-05T05:00:00.000Z
Last candle epoch 1564984800000 2019-08-05T06:00:00.000Z
Last candle epoch 1564988400000 2019-08-05T07:00:00.000Z
Last candle epoch 1564992000000 2019-08-05T08:00:00.000Z
Last candle epoch 1564995600000 2019-08-05T09:00:00.000Z
Last candle epoch 1564999200000

Last candle epoch 1565456400000 2019-08-10T17:00:00.000Z
Last candle epoch 1565460000000 2019-08-10T18:00:00.000Z
Last candle epoch 1565463600000 2019-08-10T19:00:00.000Z
Last candle epoch 1565467200000 2019-08-10T20:00:00.000Z
Last candle epoch 1565470800000 2019-08-10T21:00:00.000Z
Last candle epoch 1565474400000 2019-08-10T22:00:00.000Z
Last candle epoch 1565478000000 2019-08-10T23:00:00.000Z
Last candle epoch 1565481600000 2019-08-11T00:00:00.000Z
Last candle epoch 1565485200000 2019-08-11T01:00:00.000Z
Last candle epoch 1565488800000 2019-08-11T02:00:00.000Z
Last candle epoch 1565492400000 2019-08-11T03:00:00.000Z
Last candle epoch 1565496000000 2019-08-11T04:00:00.000Z
Last candle epoch 1565499600000 2019-08-11T05:00:00.000Z
Last candle epoch 1565503200000 2019-08-11T06:00:00.000Z
Last candle epoch 1565506800000 2019-08-11T07:00:00.000Z
Last candle epoch 1565510400000 2019-08-11T08:00:00.000Z
Last candle epoch 1565514000000 2019-08-11T09:00:00.000Z
Last candle epoch 1565517600000

Last candle epoch 1566003600000 2019-08-17T01:00:00.000Z
Last candle epoch 1566007200000 2019-08-17T02:00:00.000Z
Last candle epoch 1566010800000 2019-08-17T03:00:00.000Z
Last candle epoch 1566014400000 2019-08-17T04:00:00.000Z
Last candle epoch 1566018000000 2019-08-17T05:00:00.000Z
Last candle epoch 1566021600000 2019-08-17T06:00:00.000Z
Last candle epoch 1566025200000 2019-08-17T07:00:00.000Z
Last candle epoch 1566028800000 2019-08-17T08:00:00.000Z
Last candle epoch 1566032400000 2019-08-17T09:00:00.000Z
Last candle epoch 1566036000000 2019-08-17T10:00:00.000Z
Last candle epoch 1566039600000 2019-08-17T11:00:00.000Z
Last candle epoch 1566043200000 2019-08-17T12:00:00.000Z
Last candle epoch 1566046800000 2019-08-17T13:00:00.000Z
Last candle epoch 1566050400000 2019-08-17T14:00:00.000Z
Last candle epoch 1566054000000 2019-08-17T15:00:00.000Z
Last candle epoch 1566057600000 2019-08-17T16:00:00.000Z
Last candle epoch 1566061200000 2019-08-17T17:00:00.000Z
Last candle epoch 1566064800000

Last candle epoch 1566522000000 2019-08-23T01:00:00.000Z
Last candle epoch 1566525600000 2019-08-23T02:00:00.000Z
Last candle epoch 1566529200000 2019-08-23T03:00:00.000Z
Last candle epoch 1566532800000 2019-08-23T04:00:00.000Z
Last candle epoch 1566536400000 2019-08-23T05:00:00.000Z
Last candle epoch 1566540000000 2019-08-23T06:00:00.000Z
Last candle epoch 1566543600000 2019-08-23T07:00:00.000Z
Last candle epoch 1566547200000 2019-08-23T08:00:00.000Z
Last candle epoch 1566550800000 2019-08-23T09:00:00.000Z
Last candle epoch 1566554400000 2019-08-23T10:00:00.000Z
Last candle epoch 1566558000000 2019-08-23T11:00:00.000Z
Last candle epoch 1566561600000 2019-08-23T12:00:00.000Z
Last candle epoch 1566565200000 2019-08-23T13:00:00.000Z
Last candle epoch 1566568800000 2019-08-23T14:00:00.000Z
Last candle epoch 1566572400000 2019-08-23T15:00:00.000Z
Last candle epoch 1566576000000 2019-08-23T16:00:00.000Z
Last candle epoch 1566579600000 2019-08-23T17:00:00.000Z
Last candle epoch 1566583200000

Last candle epoch 1567040400000 2019-08-29T01:00:00.000Z
Last candle epoch 1567044000000 2019-08-29T02:00:00.000Z
Last candle epoch 1567047600000 2019-08-29T03:00:00.000Z
Last candle epoch 1567051200000 2019-08-29T04:00:00.000Z
Last candle epoch 1567054800000 2019-08-29T05:00:00.000Z
Last candle epoch 1567058400000 2019-08-29T06:00:00.000Z
Last candle epoch 1567062000000 2019-08-29T07:00:00.000Z
Last candle epoch 1567065600000 2019-08-29T08:00:00.000Z
Last candle epoch 1567069200000 2019-08-29T09:00:00.000Z
Last candle epoch 1567072800000 2019-08-29T10:00:00.000Z
Last candle epoch 1567076400000 2019-08-29T11:00:00.000Z
Last candle epoch 1567080000000 2019-08-29T12:00:00.000Z
Last candle epoch 1567083600000 2019-08-29T13:00:00.000Z
Last candle epoch 1567087200000 2019-08-29T14:00:00.000Z
Last candle epoch 1567090800000 2019-08-29T15:00:00.000Z
Last candle epoch 1567094400000 2019-08-29T16:00:00.000Z
Last candle epoch 1567098000000 2019-08-29T17:00:00.000Z
Last candle epoch 1567101600000

In [20]:
output_file = '{}/data/ccxt/{}_{}_{}.csv'.format(home_path,symbol_, market, timewindow)

In [21]:
full_header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume','N_buy','N_sell','buy_amount_avg','sell_amount_avg','buy_amount_std','sell_amount_std','price_avg','price_std','cost_avg','cost_std']

In [22]:
df = pd.DataFrame(np_df, columns = full_header )

In [23]:
print(df.shape)

(300000, 16)


In [25]:
output_file = '{}/data/ccxt/raw_{}_{}_{}.csv'.format(home_path,symbol_, market, timewindow)
print(output_file)
df.to_csv(output_file)

/home/nam/data/ccxt/raw_ETH_USDT_binance_1h.csv


In [26]:
df = df.dropna()
print(df.shape)

(299982, 16)


In [27]:
# df['MA_Close_240'] = df.rolling(240).mean().Close

In [28]:
output_file = '{}/data/ccxt/{}_{}_{}.csv'.format(home_path,symbol_, market, timewindow)
print(output_file)

/home/nam/data/ccxt/ETH_USDT_binance_1h.csv


In [29]:
df.to_csv(output_file)